In [97]:
library(tidyverse)
library(repr)
library(tidymodels)
library(readxl)
set.seed(9999)

In [136]:
df_default <- read_excel('data/default-cc-data.xls')  %>%
    select(-c(X1:X11))%>%
    rename(client_id = "...1", sep_bill = X12 , aug_bill= X13 , july_bill= X14,june_bill= X15, may_bill= X16, april_bill=X17,sep_paid= X18 ,aug_paid= X19, july_paid= X20, june_paid= X21 , may_paid= X22 ,april_paid= X23, default_status = Y)%>%
    slice(2:30001) %>%
    mutate_if(is.character, as.numeric) %>%
    mutate(client_id = as.character(client_id)) %>%
    mutate(default_status = case_when(default_status =="1"~"yes", default_status== "0"~"no"))%>%
    mutate(default_status = as.factor(default_stats))
        
 

head(df_default)

New names:
* `` -> ...1



ERROR: Error: Problem with `mutate()` input `default_status`.
[31m✖[39m object 'default_stats' not found
[34mℹ[39m Input `default_status` is `as.factor(default_stats)`.
